In [1]:
import os
import re
import configparser
import pickle

import pandas as pd
import numpy as np

from operator import iadd, isub

from sqlalchemy import create_engine, text

In [2]:
class Character:
    def __init__(self, game_class, race):
        self.item_class_map = {'consumable': 0, 'container': 1, 'weapon': 2, 'armor': 4, 'reagent': 5, 
                               'projectile': 6, 'trade good': 7, 'recipe': 9, 'quiver': 11, 'quest': 12, 
                               'key': 13, 'miscellaneous': 15}

        self.item_subclass_map = {'cloth': 1, 'leather': 2, 'mail': 3, 'plate': 4}

        self.item_quality_map = {'poor': 0, 'common': 1, 'uncommon': 2, 
                                 'rare': 3, 'epic': 4, 'legendary': 5}

        self.item_type_map = {'head': [1], 'neck': [2], 'shoulders': [3], 'chest': [4, 5, 20], 
                              'waist': [6], 'legs': [7], 'feet': [8], 'wrists': [9], 
                              'hands': [10], 'finger': [11], 'trinket': [12], 
                              'one-hand': [13, 21], 'shield': [14], 'ranged': [15], 
                              'back': [16], 'two_hand': [17], 'offhand': [22], 'thrown': [25], 
                              'gun': [26], 'bow': [15], 'left-hand': [22], 'relic': 28}

        self.stat_reverse_map = {1: 'health', 3: 'agility', 4: 'strenght', 
                                 5: 'intellect', 6: 'spirit', 7: 'stamina'}

        self.stat_map = {'health': 1, 'agility': 3, 'strenght': 4, 
                         'intellect': 5, 'spirit': 6, 'stamina': 7}

        self.spell_map = {'on use': 0, 'on equip': 1, 'chance on hit': 2, 
                          'soulstone': 4, 'on use without delay': 5}

        self.bounding_map = {'no binding': 0, 'bind on pickup': 1, 'bind on equip': 2, 
                             'bind on use': 3 ,'quest item': 4}

        self.damage_map = {'physical': 0, 'holy': 1, 'fire': 2, 'nature': 3, 
                           'frost': 4, 'shadow': 5, 'arcane': 6}

        self.race_map = {'human': 1, 'orc': 2, 'dwarf': 3, 'elf': 4, 
                         'undead': 5, 'tauren': 6, 'gnome': 7, 'troll': 8}

        self.class_map = {'warrior': 1, 'paladin': 2, 'hunter': 3, 'rogue': 4, 'prist': 5, 
                          'shaman': 7, 'mage': 8, 'warlock': 9, 'druid': 11}
        
        self.game_class = self.valid_key(game_class, self.class_map)
        self.race = self.valid_key(race, self.race_map)
        
        self.engine = self.connect()
        self.items = pd.read_sql_query(""" 
        SELECT 
          entry AS id, name, AllowableClass, InventoryType, subclass, Quality, bonding, 
          armor, holy_res, fire_res, nature_res, frost_res, shadow_res, arcane_res,
          stat_type1, stat_value1, 
          stat_type2, stat_value2, 
          stat_type3, stat_value3, 
          stat_type4, stat_value4,
          stat_type5, stat_value5,
          dmg_min1, dmg_max1, dmg_type1,
          dmg_min2, dmg_max2, dmg_type2,
          dmg_min3, dmg_max3, dmg_type3,
          delay,
          spelltrigger_1, s1.SpellName AS sp1, s1.EffectBasePoints1 AS spb1, 
          spelltrigger_2, s2.SpellName AS sp2, s2.EffectBasePoints1 AS spb2, 
          spelltrigger_3, s3.SpellName AS sp3, s3.EffectBasePoints1 AS spb3
        FROM item_template
          LEFT JOIN spell_template as s1 ON spellid_1 = s1.Id
          LEFT JOIN spell_template as s2 ON spellid_2 = s2.Id
          LEFT JOIN spell_template as s3 ON spellid_3 = s3.Id
        """, self.engine)
        
        self.base_hp_mana = pd.read_sql_query("""
        SELECT *
        FROM player_classlevelstats
        WHERE level = 60 and class = %(cls)s;
        """, self.engine, params={'cls': self.class_map[self.game_class]})
        
        self.base_stats = pd.read_sql_query("""
        SELECT *
        FROM player_levelstats
        WHERE level = 60 and class = %(cls)s and race = %(race)s;
        """, self.engine, params={'cls': self.class_map[self.game_class], 
                                  'race': self.race_map[self.race]})
        
        # main stats
        self.sta = self.base_stats['sta'].values[0]
        self.str = self.base_stats['str'].values[0]
        self.inte = self.base_stats['inte'].values[0]
        self.spi = self.base_stats['spi'].values[0]
        self.agi = self.base_stats['agi'].values[0]
        
        # resist
        self.holy_res = 0
        self.fire_res = 0
        self.nature_res = 0
        self.frost_res = 0
        self.shadow_res = 0
        self.arcane_res = 0
        
        # base values of stats
        self.bonus_hp = 0
        self.base_armor = 0
        self.base_attack_power = 0
        self.base_spell_crit = 0
        self.base_crit = 0
        self.base_dodge = 0
        self.mana_reg_bonus = {'paladin': 15, 'hunter': 15, 'warlock': 15, 'druid': 15,
                               'prist': 12.5, 'mage': 12.5, 
                               'shaman': 17,
                               'warrior': 0, 'rogue': 0}[self.game_class]  
        
        self.hp = 0
        self.mana = 0
        self.armor = 0
        self.melee_attack_power = 0
        self.range_attack_power = 0
        self.spell_power = 0
        self.spell_holy_power = 0
        self.spell_fire_power = 0
        self.spell_nature_power = 0
        self.spell_frost_power = 0
        self.spell_shadow_power = 0
        self.spell_arcane_power = 0
        self.healing_power = 0
        self.crit = 0
        self.spell_crit = 0
        self.dodge = 0
        self.parry = 0
        self.defence = 0
        self.mana_reg = 0
        self.hit_chance = 0
        
        self.calculate_stats()
        
        self.items_on = {item: None for item in self.item_type_map}
        
    def valid_key(self, key, mapper):
        # check if key is valid
        if key in mapper:
            return key
        else: 
            print('Valid keys are: ', mapper.keys())
            raise KeyError('Invalid key: {}'.format(key))
    
    def calculate_stats(self):
        self.hp = self.base_hp_mana['basehp'].values[0] + self.sta * 10 + self.bonus_hp
        self.mana = self.base_hp_mana['basemana'].values[0] + self.inte * 15
        self.armor = self.agi * 2 + self.base_armor
        
        # melee attack power from strenght 
        if self.game_class in ['hunter', 'mage', 'prist', 'rogue', 'warlock']:
            self.melee_attack_power = self.base_attack_power + self.sta
        else:
            self.melee_attack_power = self.base_attack_power + self.sta * 2
        
        # range attack power from agility 
        if self.game_class in ['rogue', 'warrior']:
            self.range_attack_power = self.base_attack_power + self.agi
        elif self.game_class == 'hunter':
            self.range_attack_power = self.base_attack_power + self.agi * 2
            
        # melee attack power from agility 
        if self.game_class in ['rogue', 'druid', 'hunter']:
            self.melee_attack_power += self.agi
            
        # crit from agility
        if self.game_class in ['druid', 'paladin', 'shaman', 'warrior']:
            self.crit = self.base_crit + self.agi / 20
        elif self.game_class == 'rogue':
            self.crit = self.base_crit + self.agi / 29            
        elif self.game_class == 'hunter':
            self.crit = self.base_crit + self.agi / 53       
        
        # spell crit from intellect
        if self.game_class == 'paladin':
            self.spell_crit = self.base_spell_crit + self.inte / 54
        else:
            self.spell_crit = self.base_spell_crit + self.inte / 60
            
        # mana reg
        if self.game_class in ['druid', 'paladin', 'warlock', 'hunter', 'shaman']:
            self.mana_reg = self.mana_reg_bonus + self.spi / 5
        elif self.game_class in ['mage', 'prist']:
            self.mana_reg = self.mana_reg_bonus + self.spi / 4
        
        # dodge from agility
        if self.game_class == 'rogue':
            self.dodge = self.base_dodge + self.agi / 14.5            
        elif self.game_class == 'hunter':
            self.dodge = self.base_dodge + self.agi / 26
        else:
            self.dodge = self.base_dodge + self.agi / 20     
        
            
    def wear_item(self, slot, ids):
        # correctness of slot
        slot = self.valid_key(slot, self.item_type_map)
        
        # check if item with such id exist
        temp = self.items.loc[self.items['id'] == ids]
        if temp.shape[0] == 0:
            raise KeyError('No item with such id.')
        
        # check if character can wear it
        if (temp['AllowableClass'].values[0] == -1) or \
        (temp['AllowableClass'].values[0] == self.class_map[self.game_class]):
            pass
        else:
            raise KeyError('Can\'t be used by your class.')
            
        # remove old item
        if self.items_on[slot] is not None:
            self.add_remove_stats('sub', self.items.loc[self.items['id'] == self.items_on[slot]])
            self.items_on[slot] = None
        
        # add stats from new item
        self.add_remove_stats('add', temp)
        
        # update dict
        self.items_on[slot] = ids
    
    def add_remove_stats(self, operation, temp):
        if operation == 'sub':
            operator = isub
        elif operation == 'add':
            operator = iadd
            
        # main stats
        for i in range(1, 6):
            stat_type = temp['stat_type{}'.format(i)].values[0]
            value = temp['stat_value{}'.format(i)].values[0]
            if stat_type == 1:
                self.bonus_hp = operator(self.bonus_hp, value)
            elif stat_type == 3:
                self.agi = operator(self.agi, value)
            elif stat_type == 4:
                self.str = operator(self.str, value)
            elif stat_type == 5:
                self.inte = operator(self.inte, value)
            elif stat_type == 6:
                self.spi = operator(self.spi, value)
            elif stat_type == 7:
                self.sta = operator(self.sta, value)  
                
        # armor
        self.base_armor = operator(self.base_armor, temp['armor'].values[0])
        
        # resist
        for string, resist_type in zip(['holy_res', 'fire_res', 'nature_res', 
                                        'frost_res', 'shadow_res', 'arcane_res'],
                                       [self.holy_res, self.fire_res, self.nature_res, 
                                        self.frost_res, self.shadow_res, self.arcane_res]):
            value = temp[string].values[0]
            resist_type = operator(resist_type, value)
        
        
        # green bonuses
        for i in range(1, 4):
            bonus_type = temp['sp{}'.format(i)].values[0]
            value = temp['spb{}'.format(i)].values[0]
            # if None no need to check stats in other columns 
            if bonus_type is None:
                    break
            for bonus, stat in zip(['Increase Spell Dam', 'Increase Fire Dam', 'Increase Shadow Dam', 
                                    'Increase Nature Dam', 'Increase Frost Dam', 'Increase Holy Dam', 
                                    'Increase Arcane Dam', 'Increase Healing', 'Increased Critical', 
                                    'Increased Critical Spell', 'Increased Mana Regen', 'Increased Defense', 
                                    'Increased Dodge', 'Increased Parry', 'Attack Power', 'Increased Hit Chance'],
                                   [self.spell_power, self.spell_fire_power, self.spell_shadow_power, 
                                    self.spell_nature_power, self.spell_frost_power, self.spell_holy_power, 
                                    self.spell_arcane_power, self.healing_power, self.base_crit, 
                                    self.base_spell_crit, self.mana_reg_bonus, self.defence,
                                   self.base_dodge, self.parry, self.base_attack_power, self.hit_chance]):
                # check if  within available bonuses
                if re.search('{}( \d*)*'.format(bonus), bonus_type):
                    # if bonus has it value in the end of string
                    if re.search('{}( \d*)*'.format(bonus), bonus_type).group(1):
                        value = int(re.search('{}( \d*)*'.format(bonus), bonus_type).group(1).strip())
                    stat = operator(stat, value)    
        
        # recalculate stats
        self.calculate_stats()
    
    def remove_item(self, slot):
        slot = self.valid_key(slot, self.item_type_map)
        # old item to remove
        temp = self.items.loc[self.items['id'] == self.items_on[slot]]
        
        # update stats
        self.add_remove_stats('sub', temp)
        
        # update dict
        self.items_on[slot] = None
        
    def summary(self):
        return {'hp': self.hp, 'mana': self.mana, 'mana_reg': self.mana_reg, 
                'stamina': self.sta, 'strength': self.str, 'intellect': self.inte, 
                'agility': self.agi,'spirit': self.spi, 
                'holy_resist': self.holy_res, 'fire_resist': self.fire_res, 
                'nature_resist': self.nature_res, 'frost_resist': self.frost_res,
                'shadow_resist': self.shadow_res, 'arcane_resist': self.arcane_res,
                'max_magic_reduction': self.magical_damage_reduction(),
                'armor': self.armor, 'physical_reduction': self.physical_damage_reduction(),
                'melee_ap': self.melee_attack_power, 'range_ap': self.range_attack_power,
                'sp': self.spell_power, 'holy_sp': self.spell_holy_power, 
                'fire_sp': self.spell_fire_power, 'nature_sp': self.spell_nature_power,
                'frost_sp': self.spell_frost_power, 'shadow_sp': self.spell_shadow_power,
                'arcane_sp': self.spell_arcane_power, 'healing_power': self.healing_power,
                'crit': self.crit, 'spell_crit': self.spell_crit,
                'dodge': self.dodge, 'parry': self.parry, 'defence': self.defence,
                'hit_chance': self.hit_chance}

    def physical_damage_reduction(self, attacker_level=60):
        return self.armor / (self.armor + (467.5 * attacker_level - 22167.5))        

    def magical_damage_reduction(self, caster_level=60):
        resist = max([self.holy_res, self.fire_res, self.nature_res, 
                      self.frost_res, self.shadow_res, self.arcane_res])
        return (resist / (5 * caster_level)) * 0.75
    
    def connect(self):
        # read config
        config = configparser.ConfigParser()
        config.read('main.ini')
        # connect to database
        engine = create_engine('mysql+pymysql://{}:{}@localhost/{}'.format(config['SQL']['username'], 
                                                                           config['SQL']['password'], 
                                                                           config['SQL']['database']))
        return engine
    
    def save(self, name, path='./characters'):
        'Save class instance and delete connection to db for security reason'
        # check if connection is established
        if 'engine' in s.__dict__:
            del self.engine
        pickle.dump(self, open(os.path.join(path, name), 'wb'))
        
    @staticmethod    
    def load(name, path='./characters'):
        'Load class instance and create new connection to database'
        inst = pickle.load(open(os.path.join(path, name), 'rb'))
        inst.engine = inst.connect()
        return inst

In [3]:
s = Character(game_class='warrior', race='orc')

In [4]:
s.save('example')

In [5]:
b = Character.load('example')

In [6]:
b.game_class

'warrior'

In [7]:
s.summary()

{'agility': 77,
 'arcane_resist': 0,
 'arcane_sp': 0,
 'armor': 154,
 'crit': 3.85,
 'defence': 0,
 'dodge': 3.85,
 'fire_resist': 0,
 'fire_sp': 0,
 'frost_resist': 0,
 'frost_sp': 0,
 'healing_power': 0,
 'hit_chance': 0,
 'holy_resist': 0,
 'holy_sp': 0,
 'hp': 2809,
 'intellect': 27,
 'mana': 405,
 'mana_reg': 0,
 'max_magic_reduction': 0.0,
 'melee_ap': 224,
 'nature_resist': 0,
 'nature_sp': 0,
 'parry': 0,
 'physical_reduction': 0.025511471879400314,
 'range_ap': 77,
 'shadow_resist': 0,
 'shadow_sp': 0,
 'sp': 0,
 'spell_crit': 0.45,
 'spirit': 48,
 'stamina': 112,
 'strength': 123}

In [8]:
s.game_class

'warrior'

In [9]:
s.hp, s.mana, s.armor

(2809, 405, 154)

In [10]:
s.items.loc[s.items['id'] == 940].T

,216
id,940
name,Robes of Insight
AllowableClass,-1
InventoryType,20
subclass,1
Quality,4
bonding,2
armor,74
holy_res,0
fire_res,0


In [11]:
s.wear_item('legs', 940)

In [12]:
s.hp, s.mana, s.armor

(2809, 780, 228)

In [13]:
s.remove_item('legs')

In [14]:
s.hp, s.mana, s.armor

(2809, 405, 154)

In [15]:
s.items.loc[s.items['Quality'] == 4, ['sp1', 'spb1', 'sp2', 'spb2', 'sp3', 'spb3']]

,sp1,spb1,sp2,spb2,sp3,spb3
60,Destiny,199.0,None,NaN,None,NaN
127,Rend,11.0,None,NaN,None,NaN
128,Frostbolt,-51.0,None,NaN,None,NaN
129,Wrath,89.0,None,NaN,None,NaN
130,None,NaN,None,NaN,None,NaN
141,Lifestone Healing,299.0,Lifestone Regeneration,9.0,None,NaN
170,Increased Critical 1,0.0,Attack Power 20,19.0,Undead Slayer 30,29.0
171,Increased Defense,4.0,None,NaN,None,NaN
172,Faerie Fire,-101.0,None,NaN,None,NaN
173,Fireball,154.0,None,NaN,None,NaN
